### Monte Carlo with Exploring Starts for estimating an optimal Blackjack policy

This corresponds to the algorithm described in Chapter 5.3 of [_Reinforcement Learning: An Introduction_](http://incompleteideas.net/book/the-book-2nd.html), by Sutton and Barto

In [47]:
# Dependencies
from typing import Any, Tuple

import gym

from rl_agents.abstract import IBaseAgent
from rl_agents.blackjack.MonteCarloAgent import MonteCarloAgent

### Setup Blackjack agent and environment

In [48]:
env = gym.make('Blackjack-v0')
agent = MonteCarloAgent(action_space=env.action_space,
                        obs_space=env.observation_space)

In [49]:
print(agent.rewards)

[[[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0. 0.]
  [0

### Setup Blackjack Experiment

In [34]:

for i_episode in range(1):
    observation = env.reset()
    done = False
    t = 0
    while not done:
        print(observation)
        action = env.action_space.sample()
        print("action={}".format(action))
        observation, reward, done, info = env.step(action)
        print("observation={}".format(action))
        print("reward={}".format(action))
        print("done={}".format(action))
        print("info={}".format(action))
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
            
        t += 1

(9, 4, False)
action=0
observation=0
reward=0
done=0
info=0
Episode finished after 1 timesteps


In [36]:
env.action_space

TypeError: 'Discrete' object is not iterable